# Examples from [`README.md`](../../README.md)

The readme contains a number of generalized examples. This presents each,
followed by specific code similar to it.

This notebook does not reproduce most of the readme, only (some of) the Python
code in it. This also does not replace the unit tests (in [`tests/`](../../tests/)). Showing that the *library* behaves correctly in all important respect is *not* one of the goals of this notebook. Showing that the the examples in the readme appear correct and are basically reasonable *is* one of the goals of this notebooks.

Major section headings in this notebook are links to corresponding section in
the readme.

In [1]:
import contextlib
from dataclasses import InitVar, dataclass
import glob
import io
import os
from pathlib import Path
import pprint
import unittest
from unittest.mock import ANY, Mock

import subaudit

In [2]:
# Support both VS Code and JupyterLab, which give notebooks different CWDs.
if not Path('README.md').exists():
    grandparent = Path().absolute().parent.parent
    assert (grandparent / 'README.md').exists()
    os.chdir(grandparent)

In [3]:
def get_h1(file: io.FileIO) -> str:
    """Get a Markdown document heading. Unreliable but OK for demonstration."""
    return next(line[1:].strip() for line in file if line.startswith('#'))

## [The `subaudit.listening` context manager](../../README.md#the-subauditlistening-context-manager)

### Defining a function and using it as a listener

Generalized code from the readme:

```python
import subaudit

def listen_open(path, mode, flags):
    ...  # Handle the event.

with subaudit.listening('open', listen_open):
    ...  # Do something that may raise the event.
```

Specific runnable code example:

In [4]:
def listen_open(path: str, mode: str, flags: int) -> None:
    print(f'{path=}, {mode=}, {flags=}')

with subaudit.listening('open', listen_open):
    with open('README.md', encoding='utf-8') as file:
        print(get_h1(file))

path='README.md', mode='r', flags=524288
subaudit: Subscribe and unsubscribe for specific audit events


### Using a `Mock` as a listener

Generalized code from the readme:

```python
from unittest.mock import ANY, Mock
import subaudit

with subaudit.listening('open', Mock()) as listener:
    ...  # Do something that may raise the event.

listener.assert_any_call('/path/to/file.txt', 'r', ANY)
```

Specific runnable code example:

In [5]:
with subaudit.listening('open', Mock()) as listener:
    with open('README.md', encoding='utf-8') as file:
        print(get_h1(file))

# This should succeed silently.
listener.assert_any_call('README.md', 'r', ANY)

# Print the calls (which is probably just one call).
listener.mock_calls

subaudit: Subscribe and unsubscribe for specific audit events


[call('README.md', 'r', 524288)]

## [The `subaudit.extracting` context manager](../../README.md#the-subauditextracting-context-manager)

Generalized code from the readme:

```python
from dataclasses import InitVar, dataclass
import subaudit

@dataclass(frozen=True)
class PathAndMode:  # Usually str and int. See notebooks/open_event.ipynb.
    path: str
    mode: str
    flags: InitVar = None  # Opt not to record this argument.

with subaudit.extracting('open', PathAndMode) as extracts:
    ...  # Do something that may raise the event.

assert PathAndMode('/path/to/file.txt', 'r') in extracts
```

Specific runnable code example:

In [6]:
@dataclass(frozen=True)
class PathAndMode:  # Usually str and int. See notebooks/open_event.ipynb.
    path: str
    mode: str
    flags: InitVar = None  # Opt not to record this argument.

with subaudit.extracting('open', PathAndMode) as extracts:
    with open('README.md', encoding='utf-8') as file:
        print(get_h1(file))

# This should succeed silently.
assert PathAndMode('README.md', 'r') in extracts

# Print the extracted data.
extracts

subaudit: Subscribe and unsubscribe for specific audit events


[PathAndMode(path='README.md', mode='r')]

## [`subaudit.subscribe` and `subaudit.unsubscribe`](../../README.md#subauditsubscribe-and-subauditunsubscribe)

### Showing `subscribe` and `unsubscribe` with `try`-`finally`

Generalized code from the readme:

```python
import subaudit

def listen_open(path, mode, flags):
    ...  # Handle the event.

subaudit.subscribe('open', listen_open)
try:
    ...  # Do something that may raise the event.
finally:
    subaudit.unsubscribe('open', listen_open)
```

Specific runnable code example:

In [7]:
def listen_open(path: str, mode: str, flags: int) -> None:
    print(f'{path=}, {mode=}, {flags=}')

subaudit.subscribe('open', listen_open)
try:
    with open('README.md', encoding='utf-8') as file:
        print(get_h1(file))
finally:
    subaudit.unsubscribe('open', listen_open)

path='README.md', mode='r', flags=524288
subaudit: Subscribe and unsubscribe for specific audit events


### Suppressing unsubscribe `ValueError`

Generalized code from the readme:

```python
with contextlib.suppress(ValueError):
    subaudit.unsubscribe('glob.glob', possibly_subscribed_listener)
```

Specific runnable code example:

In [8]:
listener1 = Mock()
listener2 = Mock()
listener3 = Mock()  # In this example, we don't actually subscribe this.
try:
    # Normally we acquire the resource before entering the try-block, but here
    # the point is that we're freeing the resource (actually, resources) in a
    # way that tolerates if they have never actually been acquired.
    for listener in listener1, listener2:
        subaudit.subscribe('glob.glob', listener)
finally:
    for listener in listener1, listener2, listener3:
        with contextlib.suppress(ValueError):
            subaudit.unsubscribe('glob.glob', listener)

## [Nesting](../../README.md#nesting)

### Three `listening` contexts

Generalized code from the readme:

```python
from unittest.mock import Mock, call

listen_to = Mock()  # Let us assert calls to child mocks in a specific order.

with subaudit.listening('open', print):  # Print all open events' arguments.
    with subaudit.listening('open', listen_to.open):  # Log opening.
        with subaudit.listening('glob.glob', listen_to.glob):  # Log globbing.
            ...  # Do something that may raise the events.

assert listen_to.mock_calls == ...  # Assert a specific order of calls.
```

Specific runnable code example:

In [9]:
listen_to = Mock()  # Let us assert calls to child mocks in a specific order.

with subaudit.listening('open', print):  # Print all open events' arguments.
    with subaudit.listening('open', listen_to.open):  # Log opening.
        with subaudit.listening('glob.glob', listen_to.glob):  # Log globbing.
            paths = glob.glob('README.*')
            with open(paths[0], encoding='utf-8') as file:
                print(get_h1(file))

# Show the calls.
listen_to.mock_calls

README.md r 524288
subaudit: Subscribe and unsubscribe for specific audit events


[call.glob('README.*', False), call.open('README.md', 'r', 524288)]

### Both `listening` and `extracting` contexts

Generalized code from the readme:

```python
from unittest.mock import Mock, call

def extract(*args):
    return args

with (
    subaudit.extracting('pathlib.Path.glob', extract) as glob_extracts,
    subaudit.listening('pathlib.Path.glob', Mock()) as glob_listener,
    subaudit.extracting('pathlib.Path.rglob', extract) as rglob_extracts,
    subaudit.listening('pathlib.Path.rglob', Mock()) as rglob_listener,
):
    ...  # Do something that may raise the events.

# Assert something about, or otherwise use, the mocks glob_listener and
# rglob_listener, as well as the lists glob_extracts and rglob_extracts.
...
```

Specific runnable code example (using an uglier syntax, because I don't want this notebook to require Python 3.10):

In [10]:
def extract(*args):
    return args

with \
    subaudit.extracting('pathlib.Path.glob', extract) as glob_extracts, \
    subaudit.listening('pathlib.Path.glob', Mock()) as glob_listener, \
    subaudit.extracting('pathlib.Path.rglob', extract) as rglob_extracts, \
    subaudit.listening('pathlib.Path.rglob', Mock()) as rglob_listener \
:
    print('Non-recursive:')
    pprint.pp(list(Path().glob('*.ipynb')))
    print()
    print('Recursive:')
    pprint.pp(list(Path().rglob('*.ipynb')))
    print()
    print('Non-recursive again (still empty):')
    pprint.pp(list(Path().glob('*.ipynb')))

Non-recursive:
[]

Recursive:
[PosixPath('examples/notebooks/from_readme.ipynb'),
 PosixPath('examples/notebooks/input_events.ipynb'),
 PosixPath('examples/notebooks/open_event.ipynb')]

Non-recursive again (still empty):
[]


In [11]:
glob_listener.mock_calls

[call(PosixPath('.'), '*.ipynb'), call(PosixPath('.'), '*.ipynb')]

In [12]:
rglob_listener.mock_calls

[call(PosixPath('.'), '*.ipynb')]

In [13]:
glob_extracts

[(PosixPath('.'), '*.ipynb'), (PosixPath('.'), '*.ipynb')]

In [14]:
rglob_extracts

[(PosixPath('.'), '*.ipynb')]

*The [Specialized usage](../../README.md#specialized-usage) section contains no code blocks, so this notebook contains no corresponding section.*

## [Functions related to compatibility](../../README.md#functions-related-to-compatibility)

Only one of these has an example.

### `@subaudit.skip_if_unavailable`

Generalized code from the readme:

```python
import unittest
from unittest.mock import ANY, Mock
import subaudit

class TestSomeThings(unittest.TestCase):
    ...

    @subaudit.skip_if_unavailable  # Skip this test if < 3.8, with a message.
    def test_file_is_opened_for_read(self):
        with subaudit.listening('open', Mock()) as listener:
            ...  # Do something that may raise the event.

        listener.assert_any_call('/path/to/file.txt', 'r', ANY)

    ...

@subaudit.skip_if_unavailable  # Skip the whole class if < 3.8, with a message.
class TestSomeMoreThings(unittest.TestCase):
    ...
```

Two specific runnable code examples follow, corresponding to the two cases.

#### Python 3.8 or later - *Not* skipped

The notebooks in this project, including this notebook, require Python 3.8. It is assumed, therefore, that the *real* situation is such that `@skip_if_unavailable` should not skip tests.

In [15]:
class TestSomeThings(unittest.TestCase):

    @subaudit.skip_if_unavailable  # Skip this test if < 3.8, with a message.
    def test_file_is_opened_for_read(self):
        with subaudit.listening('open', Mock()) as listener:
            ...  # Do something that may raise the event.

        listener.assert_any_call('/path/to/file.txt', 'r', ANY)

    def test_one_is_not_equal_to_two(self):  # Probably true even in Python 3.7.
        self.assertNotEqual(1, 2)

@subaudit.skip_if_unavailable  # Skip the whole class if < 3.8, with a message.
class TestSomeMoreThings(unittest.TestCase):
    ...